In [71]:
import pandas as pd

In [81]:
df = pd.read_csv('UN_occupation_data.csv')

In [82]:
# Drop duplicated rows
df = df.drop_duplicates()

# Filter out rows where 'Sex' is "Both Sexes"
df = df[df["Sex"] != "Both Sexes"]

# Filter out rows where 'Age' is "Total"
df = df[df["Age"] != "Total"]

# Remove text within parentheses in the 'Occupation' column
df["Occupation"] = df["Occupation"].str.replace(r"\s*\(.*?\)", "", regex=True)

# Map occupations to desired categories
occupation_mapping = {
    "Armed forces": "Armed forces",
    "Members of the Armed Forces": "Armed forces",
    "Legislators, senior officials and managers": "Office Workers",
    "Administrative and Managerial Workers": "Office Workers",
    "Administrative, Executive and Managerial Workers": "Office Workers",
    "Managers": "Office Workers",  # Ensure "Managers" is mapped to "Office Workers"
    "Professionals": "Professionals",
    "Professional, Technical and Related Workers": "Technicians",
    "Technicians and associate professionals": "Technicians",
    "Clerks": "Clerical workers",
    "Clerical support workers": "Clerical workers",
    "Service workers and shop and market sales workers": "Service and sales workers",
    "Sales Workers": "Service and sales workers",
    "Service, Sport and Recreation Workers": "Service and sales workers",
    "Service and sales workers": "Service and sales workers",
    "Skilled agricultural and fishery workers": "Agricultural workers",
    "Skilled agricultural, forestry and fishery workers": "Agricultural workers",
    "Farmers, Fishermen, Hunters, Loggers and Related Workers": "Agricultural workers",
    "Agricultural, Animal Husbandry and Forestry Workers, Fishermen and Hunters": "Agricultural workers",
    "Craft and related trades workers": "Craft workers",
    "Miners, Quarrymen and Related Workers": "Craft workers",
    "Craftsmen, Production-Process Workers, and Labourers Not Elsewhere Classified": "Craft workers",
    "Plant and machine operators and assemblers": "Plant and machine operators",
    "Plant and machine operators, and assemblers": "Plant and machine operators",
    "Production and Related Workers, Transport Equipment Operators and Labourers": "Plant and machine operators",
    "Workers in Transport and Communication Occupations": "Plant and machine operators",
    "Elementary occupations": "Elementary occupations",
    "Unknown": "Unknown",
    "Total": "Unknown",
    "Workers Not Classifiable by Occupation": "Unknown",
    "Armed forces occupations": "Armed forces",
    "Clerical and Related Workers": "Clerical Workers",
    "Clerical workers": "Clerical Workers",
    "Office Workers": "Office Workers",
    "Technicians": "Technicians",
    "Professionals": "Professionals",
    "Service Workers": "Service and sales workers",
}
df["Occupation"] = df["Occupation"].map(occupation_mapping).fillna(df["Occupation"])

# Define a function to classify the age ranges
def classify_age(age):
    if "+" in age or "-" in age or age.isnumeric():
        # Handle ranges and individual ages
        if "+" in age:
            lower_limit = int(age.split("+")[0].strip())
            if lower_limit >= 60:
                return ">60"
            elif lower_limit >= 15:
                return "15-59"
            else:
                return "<15"
        elif "-" in age:
            lower_limit, upper_limit = map(int, age.split("-"))
            if upper_limit < 15:
                return "<15"
            elif lower_limit >= 60:
                return ">60"
            else:
                return "15-59"
        elif age.isnumeric():
            age_numeric = int(age)
            if age_numeric < 15:
                return "<15"
            elif age_numeric <= 59:
                return "15-59"
            else:
                return ">60"
    return None

# Apply age classification
df["Age"] = df["Age"].apply(classify_age)

# Group by Country, Sex, Age, and Occupation, and aggregate counts
grouped = df.groupby(["Country or Area", "Sex", "Age", "Occupation"]).size().reset_index(name="Count")

# Pivot the table
pivot_table = grouped.pivot(index=["Country or Area", "Sex", "Age"], columns="Occupation", values="Count")

# Fill NaN with 0 and reset index
pivot_table = pivot_table.fillna(0).reset_index()

# Ensure the columns are formatted properly
pivot_table.columns.name = None

# Display the resulting table

pivot_table.to_csv("cleaned_UN_occupation_data.csv", index=False )
